In [14]:
from llmcam.fn_to_fc import complete, form_msgs, form_msg, tool_schema, print_msgs

In [10]:
import random

def random_generator():
    """Generate a random number between 1 and 100"""
    return random.randint(1, 100)

In [6]:
notifications = []

def send_notification(
        msg: str  # The message to send
    ):
    """Send a notification"""
    notifications.append(msg)
    return notifications

In [8]:
from threading import Thread
import time

def start_notifications_stream(
    messages: list  # Previous conversation with the user
):
    """Start the notifications stream"""
    global tools

    # Add sending notification to tool schema
    subtools = [ tool for tool in tools if tool['function']['name'] != 'start_notifications_stream' ]
    subtools.append(tool_schema(send_notification, 'send_notification'))

    # Copy the messages to avoid modifying the original list
    messages = [ message for message in messages ]

    # Define the functionality to use
    def complete_with_notifications():
        for _ in range(10):
            time.sleep(5)
            complete(messages, tools=subtools)

    # Start the notifications stream
    t = Thread(target=complete_with_notifications)
    t.start()

    return 'Notifications stream started'

In [19]:
tools = [ tool_schema(random_generator, 'random_generator') ]
tools.append(tool_schema(start_notifications_stream, 'notification'))
tools

[{'type': 'function',
  'function': {'name': 'random_generator',
   'description': 'Generate a random number between 1 and 100',
   'parameters': {'type': 'object', 'properties': {}, 'required': []},
   'metadata': {'module': '__main__', 'service': 'random_generator'}}},
 {'type': 'function',
  'function': {'name': 'start_notifications_stream',
   'description': 'Start the notifications stream',
   'parameters': {'type': 'object',
    'properties': {'messages': {'type': 'array',
      'description': 'Previous conversation with the user',
      'items': {'type': 'string'}}},
    'required': ['messages']},
   'metadata': {'module': '__main__', 'service': 'notification'}}}]

In [26]:
tools[1]['function']['parameters'] = {
    'type': 'object',
    'properties': {
        'messages': {
            'type': 'array',
            'items': {
                'type': 'object',
                'properties': {
                    'role': {
                        'type': 'string',
                        'enum': ['user', 'tool', 'system', 'assistant']
                    },
                    'content': {
                        'type': 'string'
                    }
                }
            }
        }
    }
}

In [27]:
messages = form_msgs([
    ('system', 'You are a helpful system administrator. Use the supplied tools to assist the user.'),
    ('user', 'Hello, can you generate a random number for me?'),
])
complete(messages, tools=tools)
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to assist the user.
>> User:
Hello, can you generate a random number for me?
>> Assistant:
Here is your random number: 84.


In [28]:
messages.append(form_msg('user', 'Can you notify me every time you generate a number higher than 50?'))
complete(messages, tools=tools)
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to assist the user.
>> User:
Hello, can you generate a random number for me?
>> Assistant:
Here is your random number: 84.
>> User:
Can you notify me every time you generate a number higher than 50?
>> Assistant:
The notifications stream has been started. You'll be notified every time a number higher than 50 is
generated.


In [29]:
messages

[{'role': 'system',
  'content': 'You are a helpful system administrator. Use the supplied tools to assist the user.'},
 {'role': 'user',
  'content': 'Hello, can you generate a random number for me?'},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_bXGxwsimCScazDKHQ3ddJWmn',
    'function': {'arguments': '{}', 'name': 'random_generator'},
    'type': 'function'}]},
 {'role': 'tool',
  'content': '{"random_generator": 84}',
  'tool_call_id': 'call_bXGxwsimCScazDKHQ3ddJWmn'},
 {'content': 'Here is your random number: 84.',
  'refusal': None,
  'role': 'assistant'},
 {'role': 'user',
  'content': 'Can you notify me every time you generate a number higher than 50?'},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_GkDiHPAyDLLElX8tmsjwUrgO',
    'function': {'arguments': '{"messages":[{"role":"user","content":"Please notify me every time a number higher than 50 is generated."}]}',
     'name': 'start_not

In [36]:
notifications

["A new number, 68, has been generated and it's higher than 50.",
 "A new number, 71, has been generated and it's higher than 50.",
 "A new number, 54, has been generated and it's higher than 50.",
 "A new number, 61, has been generated and it's higher than 50."]